In [1]:
import sys
import json
import re
import argparse
from datetime import datetime
from io import BytesIO
import os

import pandas as pd
import boto3
from pymongo import MongoClient


In [5]:
def load_secrets(file_path):
    if not os.path.exists(file_path):
        print(f"Error: {file_path} not found!")
        sys.exit(1)
    
    with open(file_path, 'r') as file:
        return json.load(file)
secrets = load_secrets('secrets.json')

aws_access_key_id = secrets['AWS_ACCESS_KEY_ID']
aws_secret_access_key = secrets['AWS_SECRET_ACCESS_KEY']
aws_region = secrets['AWS_REGION']

In [6]:
# Delete if not in Jupyter Notebook
if 'ipykernel' in sys.modules:
    sys.argv = ['mongoTransfert.py', 'Ichtegem']  


parser = argparse.ArgumentParser(description="Process an Excel file")
parser.add_argument(
    'files',
    nargs='*',  # Accept multiple files (or none)
    default=None,
    help='The name of the station to process. Only accepts Ichtegem or Madeleine'
)
args = parser.parse_args()

if not args.files:
    args.files = ['Ichtegem']

In [7]:
args.files[0]

'Ichtegem'

In [8]:
s3 = boto3.client('s3', 
                  aws_access_key_id=aws_access_key_id, 
                  aws_secret_access_key=aws_secret_access_key, 
                  region_name=aws_region)

In [9]:
bucket_name = 'greencoop-airbyte'
if args.files[0] == 'Ichtegem':
    file_key = "greencoop-airbyte/Ichtegem.xlsx"
elif args.files[0] == 'Madeleine':
    file_key = "greencoop-airbyte/Madeleine.xlsx"

s3_object = s3.get_object(Bucket=bucket_name, Key=file_key)
file_content = s3_object['Body'].read()

# Charger le fichier Excel avec pandas
excel_file = pd.ExcelFile(BytesIO(file_content), engine='openpyxl')

In [ ]:
# Define column renaming dictionary (with explicit units)
column_mapping = {
    "Time": "time",
    "Temperature": "temperature_°F",
    "Dew Point": "dew_point_°F",
    "Humidity": "humidity_%",
    "Wind": "wind_dir",
    "Speed": "wind_speed_mph",
    "Gust": "wind_gust_mph",
    "Pressure": "pressure_inHg",
    "Precip. Rate.": "precip_rate_in/hr",
    "Precip. Accum.": "precip_accum_in",
    "UV": "uv_index",
    "Solar": "solar_w/m²"
}

def clean_value(value):
    if isinstance(value, str):
        match = re.search(r"[-+]?\d*\.?\d+", value)  # Extract numeric part
        return float(match.group()) if match else None  # Keep None for non-numeric strings
    return value  # Return unchanged if not a string (including NaN)


# List to store processed DataFrames
dfs = []

# Loop through all sheets
for sheet_name in excel_file.sheet_names:
    # Read the current sheet
    df = excel_file.parse(sheet_name, na_values=["", "None", "NA", "NaN"])
    # Rename columns for consistency
    df.rename(columns=column_mapping, inplace=True)
    
    # Apply cleaning function to all columns (except 'time' and 'wind_dir')
    for col in df.columns:
        if col not in ["time", "wind_dir"]:  # Exclude categorical columns
            df[col] = df[col].apply(clean_value)

    df = df.dropna(how='all')

    # Convert sheet name (DDMMAAAA) to date (YYYY-MM-DD)
    date_formatted = pd.to_datetime(sheet_name, format="%d%m%y")
    
    # Add the date column
    df.insert(0, "date", date_formatted)
    df["time"] = pd.to_datetime(df["time"], format="%H:%M:%S").dt.time
    df["datetime"] = pd.to_datetime(df["date"].astype(str) + " " + df["time"].astype(str))
    df = df.drop(columns=["date", "time"])
    df = df[["datetime"] + [col for col in df.columns if col != "datetime"]]

    # Append cleaned DataFrame
    dfs.append(df)

# Combine all sheets into one DataFrame
final_df = pd.concat(dfs, ignore_index=True)

               datetime  temperature_°F  dew_point_°F  humidity_% wind_dir  \
1   2024-10-07 00:04:00            57.7          56.7        96.0       SW   
2   2024-10-07 00:09:00            57.7          56.7        96.0       SW   
3   2024-10-07 00:14:00            57.8          56.8        96.0      WSW   
4   2024-10-07 00:19:00            58.0          56.9        96.0      WSW   
5   2024-10-07 00:24:00            58.2          57.2        96.0      WSW   
..                  ...             ...           ...         ...      ...   
174 2024-10-07 14:29:00            65.2          56.3        73.0      WSW   
175 2024-10-07 14:34:00            65.5          56.3        72.0      WSW   
176 2024-10-07 14:39:00            65.7          55.5        70.0      WSW   
177 2024-10-07 14:44:00            66.4          56.1        70.0     West   
178 2024-10-07 14:49:00            66.4          55.7        68.0      WSW   

     wind_speed_mph  wind_gust_mph  pressure_inHg  precip_rate_

In [43]:
# Display unique values for each column
# for col in final_df.columns:
#    unique_values = final_df[col].unique()
#    print(f"Column: {col}")
#    print(f"Unique Values: {unique_values[:100]}")  # Display only the first 10 unique values
#    print("-" * 50)


In [44]:
final_df

,datetime,temperature_°F,dew_point_°F,humidity_%,wind_dir,wind_speed_mph,wind_gust_mph,pressure_inHg,precip_rate_in/hr,precip_accum_in,uv_index,solar_w/m²
0,2024-10-01 00:04:00,56.8,53.1,87.0,WSW,8.2,10.4,29.48,0.0,0.00,0.0,0.0
1,2024-10-01 00:09:00,56.8,52.9,87.0,WSW,7.9,9.8,29.47,0.0,0.00,0.0,0.0
2,2024-10-01 00:14:00,57.0,52.8,86.0,West,10.3,12.8,29.47,0.0,0.00,0.0,0.0
3,2024-10-01 00:19:00,57.2,52.7,85.0,WSW,9.7,12.2,29.47,0.0,0.00,0.0,0.0
4,2024-10-01 00:24:00,57.2,52.7,85.0,WSW,9.7,11.9,29.47,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1894,2024-10-07 14:29:00,65.2,56.3,73.0,WSW,3.7,4.8,29.37,0.0,0.01,4.0,414.8
1895,2024-10-07 14:34:00,65.5,56.3,72.0,WSW,3.4,4.2,29.36,0.0,0.01,4.0,416.6
1896,2024-10-07 14:39:00,65.7,55.5,70.0,WSW,4.3,6.1,29.36,0.0,0.01,3.0,338.5
1897,2024-10-07 14:44:00,66.4,56.1,70.0,West,2.9,4.1,29.36,0.0,0.01,3.0,349.6


In [ ]:
def convertToMetric(df):
    """
    Convert to metric and to other small ajustements for all data to be formated the same way
    """
    df = df.copy() 
    	
    df['dew_point_°C'] = ((df['dew_point_°F'] - 32) * 5/9).round(1)
    df['temperature_°C'] = ((df['temperature_°F'] - 32) * 5/9).round(1)
    df['wind_speed_kph'] = (df['wind_speed_mph'] * 1.60934).round(1)
    df['wind_gust_kph'] = (df['wind_gust_mph'] * 1.60934).round(1)
    df['pressure_hPa'] = (df['pressure_inHg'] * 33.8639).round(1)
    df['precip_rate_mm/hr'] = (df['precip_rate_in/hr'] * 25.4).round(1)
    df['precip_accum_mm'] = (df['precip_accum_in'] * 25.4).round(1)
    df['temperature_°C'] = ((df['temperature_°F'] - 32) * 5/9).round(1)

    df['humidity_%'] = df['humidity_%'].astype(int)

    df.drop(columns=['temperature_°F', 'wind_speed_mph', 'wind_gust_mph', 'pressure_inHg', 
                     'precip_rate_in/hr', 'precip_accum_in', 'dew_point_°F'], inplace=True)
    return df


In [94]:
final_df2 = convertToMetric(final_df)

In [ ]:
final_df2["precip_rate_mm/hr"].unique()

array([0. , 1.8, 1.3, 3. , 7.9, 6.1, 8.9, 4.8])

In [99]:
final_df2["precip_accum_mm"].unique()

array([0. , 0.3, 0.5, 0.8, 1. , 1.3, 1.5, 1.8, 3. , 3.6, 3.8, 4.1, 4.3,
       4.6, 2.3])

In [100]:
final_df2["wind_dir"].unique()   

array(['WSW', 'West', 'SW', 'SSW', 'South', 'SSE', 'East', 'SE', 'WNW',
       'ESE', 'ENE', 'NE', 'NNE', 'North', 'NNW', nan, 'NW'], dtype=object)

In [96]:
final_df2

,datetime,humidity_%,wind_dir,uv_index,solar_w/m²,dew_point_°C,temperature_°C,wind_speed_kph,wind_gust_kph,pressure_hPa,precip_rate_mm/hr,precip_accum_mm
0,2024-10-01 00:04:00,87,WSW,0.0,0.0,11.7,13.8,13.2,16.7,998.3,0.0,0.0
1,2024-10-01 00:09:00,87,WSW,0.0,0.0,11.6,13.8,12.7,15.8,998.0,0.0,0.0
2,2024-10-01 00:14:00,86,West,0.0,0.0,11.6,13.9,16.6,20.6,998.0,0.0,0.0
3,2024-10-01 00:19:00,85,WSW,0.0,0.0,11.5,14.0,15.6,19.6,998.0,0.0,0.0
4,2024-10-01 00:24:00,85,WSW,0.0,0.0,11.5,14.0,15.6,19.2,998.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1894,2024-10-07 14:29:00,73,WSW,4.0,414.8,13.5,18.4,6.0,7.7,994.6,0.0,0.3
1895,2024-10-07 14:34:00,72,WSW,4.0,416.6,13.5,18.6,5.5,6.8,994.2,0.0,0.3
1896,2024-10-07 14:39:00,70,WSW,3.0,338.5,13.1,18.7,6.9,9.8,994.2,0.0,0.3
1897,2024-10-07 14:44:00,70,West,3.0,349.6,13.4,19.1,4.7,6.6,994.2,0.0,0.3


In [97]:
# MongoDB setup
client = MongoClient("mongodb://localhost:27017/")
db = client["weather_data2"]
collection = db["weather_station2"]

In [98]:
collection.insert_many(final_df2.to_dict(orient='records'))

print("Data successfully inserted into MongoDB!")

Data successfully inserted into MongoDB!
